In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
import pandas as pd

from copy import deepcopy
import sklearn

### Import Data

In [2]:
cols = ['class','age','menopause','tumor-size',
        'inv-nodes','node-caps','deg-malig',
        'breast','breast-quad','irradiat']

df = pd.read_csv('./datasets/breast-cancer.data',names = cols)

Make all columns categorical

In [3]:
for this_col in cols[1:]:
    df[this_col] = df[this_col].astype('category')

Get mapping dict from label names to categorical indices

In [4]:
map_dicts_list = []

for col in df.columns:
    labels = np.unique(df[col])
    map_dicts_list.append(dict([[val,i] for i,val in enumerate(labels)]))
    
map_dicts_v2 = dict([[att,this_dict] for att,this_dict in zip(df.columns,map_dicts_list)])

Get instances for each class

In [5]:
print('Instance of first class:', np.sum(df['class'] == 'no-recurrence-events'))
print('Instance of second class:', np.sum(df['class'] == 'recurrence-events'))

Instance of first class: 201
Instance of second class: 85


Make normal and anomalies partitions

In [6]:
df_oc = df[df['class'] == 'no-recurrence-events']
df_oc = df_oc.drop('class',axis = 1)

In [7]:
df_anom = df[df['class'] == 'recurrence-events']
df_anom = df_anom.drop('class',axis = 1)

Make OH versions

In [8]:
df_oh = pd.get_dummies(df,columns = cols[1:])

In [9]:
oh_cols = df_oh.columns[1:]

In [10]:
df_oc_oh = df_oh[df_oh['class'] == 'no-recurrence-events']
df_oc_oh = df_oc_oh.drop('class',axis = 1)

In [11]:
df_anom_oh = df_oh[df_oh['class'] == 'recurrence-events']
df_anom_oh = df_anom_oh.drop('class',axis = 1)

### Train Trees

In [176]:
len([map_dicts_v2[label][f] for f in ys[j]])

201

In [12]:
clfs = []
xs = []
ys = []

for col in cols[1:]:
    
    this_X = df_oc_oh.loc[:,[f for f in df_oc_oh.columns if not f.startswith(col)]]
    this_y = df_oc.loc[:,col]
    
    xs.append(this_X)
    ys.append(this_y)
    
    this_clf = tree.DecisionTreeClassifier()
    this_clf.fit(this_X,this_y)
    clfs.append(this_clf)

In [13]:
#plt.figure(figsize = (15,15))
#my_tree = tree.plot_tree(clfs[0])

### Get initial weights

In [14]:
from sklearn.metrics import recall_score, roc_auc_score

Initial weights for layer 1

In [15]:
w1_init = []
w2_init = []

for i,clf in enumerate(clfs):
    w1_init.append(recall_score(ys[i],clf.predict(xs[i]),average=None))
    w2_init.append(clf.score(xs[i],ys[i]))
    
w2_init = np.array(w2_init)

In [16]:
w1_init

[array([1.        , 0.95238095, 0.93650794, 0.92957746, 0.85      ,
        0.8       ]),
 array([0.9893617 , 1.        , 0.97058824]),
 array([0.85714286, 0.85185185, 0.86956522, 0.85294118, 0.66666667,
        0.65714286, 0.66666667, 0.625     , 0.5       , 0.5       ,
        0.2       ]),
 array([1.        , 1.        , 0.66666667, 0.84210526, 1.        ,
        1.        ]),
 array([1.  , 1.  , 0.96]),
 array([0.96610169, 0.93137255, 0.85      ]),
 array([0.95145631, 0.76530612]),
 array([1.        , 0.93333333, 0.81690141, 0.77777778, 0.6       ]),
 array([1.        , 0.91891892])]

In [17]:
w2_init

array([0.91542289, 0.9800995 , 0.73134328, 0.9800995 , 0.99502488,
       0.92537313, 0.86069652, 0.85074627, 0.98507463])

In [96]:
bias_init = np.zeros((len(w2_init)))

In [98]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

### Define some network functions

In [99]:
def neuron_l1(x_prime,weights,bias,indxs):
    
    my_res = np.zeros((len(x_prime)))

    for i,this_x_prime in enumerate(x_prime):
        
        if indxs[i] >= len(weights):
            indxs[i] -= 1
        this_x_wrong = np.delete(this_x_prime,indxs[i])
        w_wrong = np.delete(weights,indxs[i])
        my_res[i] = sigmoid(this_x_prime[indxs[i]]*weights[indxs[i]]-np.mean(this_x_wrong*w_wrong)+bias)
        
    return my_res

Encoder

In [100]:
dt_y_hat = []

for i, clf in enumerate(clfs):
    dt_y_hat.append(clf.predict_proba(xs[i]))

In [156]:
w1 = deepcopy(w1_init)
w2 = deepcopy(w2_init)
b1 = deepcopy(bias_init)

lr = 0.001
iterations = 300
lrw1 = 0.001

losses = []
accs = []

y_norm = np.ones(len(df_oc_oh))

for i in range(iterations):
    
    hl1_this = np.zeros((len(df_oc_oh),len(cols[1:])))
    
    for j,x_prime in enumerate(dt_y_hat):

        x_prime_prime = x_prime*w1[j]
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys[j]]
        
        for k in np.unique(num_labels):
            
            if k >= x_prime.shape[1]:
                break
            
            indices = np.where(num_labels==k)
            this_probs = x_prime[indices]
            x_prime_this_this = x_prime[indices]
            x_prime_prime_this_this = x_prime_prime[indices]
            
            edo = neuron_l1(x_prime_this_this,w1[j],b1[j],np.ones((len(x_prime_this_this)),dtype = int)*k)
            #print(edo.shape)
            #break
            
            grad_wj = np.dot(edo*(1-edo),x_prime_this_this[:,k])
            grad_bias = np.mean(edo*(1-edo))
            
            #grad_wj = np.dot(x_prime_prime_this_this[:,k]*(1-x_prime_prime_this_this[:,k]),x_prime_this_this[:,k])
            #grad_wl = np.mean(-(1/len(np.unique(num_labels)))*np.delete(x_prime_this_this,k,axis = 1),axis = 0)
    
            w1[j][k] = w1[j][k] + lrw1*grad_wj
            b1[j] = b1[j] + lrw1*grad_bias
            #for this_indx in [f for f in np.arange(np.max(num_labels)) if f != k]:
                
            #    if this_indx < k:
            #        grad_indx = this_indx
            #    elif this_indx > k:
            #        grad_indx = this_indx-1
                    
            #    w1[j][this_indx] = w1[j][grad_indx] - lrw1*grad_wl[grad_indx]
    
    #this_loss = []
    
    for j,x_prime in enumerate(dt_y_hat):
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys[j]]
        hl1_this[:,j] = neuron_l1(x_prime,w1[j],b1[j],num_labels)
        
    this_loss = np.mean(hl1_this,axis = 0)
    #print(this_loss.shape)
    if i % 50 == 0 or i == iterations-1:
        print('Iteration {}:'.format(i+1))
        for m, loss_part in enumerate(this_loss):
            print('\tAttribute {}: {:0.4f}'.format(m+1,loss_part))

Iteration 1:
	Attribute 1: 0.6934
	Attribute 2: 0.7237
	Attribute 3: 0.6216
	Attribute 4: 0.6968
	Attribute 5: 0.7342
	Attribute 6: 0.6936
	Attribute 7: 0.6405
	Attribute 8: 0.4734
	Attribute 9: 0.7268
Iteration 51:
	Attribute 1: 0.7785
	Attribute 2: 0.8436
	Attribute 3: 0.6801
	Attribute 4: 0.8334
	Attribute 5: 0.8784
	Attribute 6: 0.7922
	Attribute 7: 0.7355
	Attribute 8: 0.4843
	Attribute 9: 0.8657
Iteration 101:
	Attribute 1: 0.8288
	Attribute 2: 0.8944
	Attribute 3: 0.7270
	Attribute 4: 0.8719
	Attribute 5: 0.9198
	Attribute 6: 0.8425
	Attribute 7: 0.7810
	Attribute 8: 0.4952
	Attribute 9: 0.9089
Iteration 151:
	Attribute 1: 0.8605
	Attribute 2: 0.9202
	Attribute 3: 0.7642
	Attribute 4: 0.8893
	Attribute 5: 0.9389
	Attribute 6: 0.8718
	Attribute 7: 0.8052
	Attribute 8: 0.5062
	Attribute 9: 0.9294
Iteration 201:
	Attribute 1: 0.8821
	Attribute 2: 0.9354
	Attribute 3: 0.7937
	Attribute 4: 0.8995
	Attribute 5: 0.9500
	Attribute 6: 0.8908
	Attribute 7: 0.8197
	Attribute 8: 0.5172
	Att

### Test

Normal Data

In [157]:
dt_norm = []

for i, clf in enumerate(clfs):
    dt_norm.append(clf.predict_proba(xs[i]))

In [158]:
hl1_this_all = np.zeros((len(df_oc_oh),len(cols[1:])))
preds = []

for j,x_prime in enumerate(dt_norm):
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys[j]]
        
        preds.append(neuron_l1(x_prime,w1[j],b1[j],num_labels))

In [159]:
for i,this_pred in enumerate(preds):
    print('Attribute {}: Score {:0.4f}'.format(i+1,np.mean(this_pred)))

Attribute 1: Score 0.9093
Attribute 2: Score 0.9522
Attribute 3: Score 0.8367
Attribute 4: Score 0.9116
Attribute 5: Score 0.9628
Attribute 6: Score 0.9137
Attribute 7: Score 0.8358
Attribute 8: Score 0.5387
Attribute 9: Score 0.9551


In [160]:
np.mean(np.array(preds))

0.8684455981977158

Anomalous Data

In [161]:
xs_anom = []
ys_anom = []

for col in cols[1:]:
    
    this_X = df_anom_oh.loc[:,[f for f in df_anom_oh.columns if not f.startswith(col)]]
    this_y = df.loc[:,col]
    
    xs_anom.append(this_X)
    ys_anom.append(this_y)

In [162]:
dt_anom = []

for i, clf in enumerate(clfs):
    dt_anom.append(clf.predict_proba(xs_anom[i]))

In [163]:
hl1_this_anom = np.zeros((len(df_anom_oh),len(cols[1:])))
preds_anom = []

for j,x_prime in enumerate(dt_anom):
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys_anom[j]]
        
        preds_anom.append(neuron_l1(x_prime,w1[j],b1[j],num_labels))

In [164]:
for i,this_pred in enumerate(preds_anom):
    print('Attribute {}: Score {:0.4f}'.format(i+1,np.mean(this_pred)))

Attribute 1: Score 0.6013
Attribute 2: Score 0.5484
Attribute 3: Score 0.6555
Attribute 4: Score 0.8197
Attribute 5: Score 0.7088
Attribute 6: Score 0.5300
Attribute 7: Score 0.5282
Attribute 8: Score 0.5857
Attribute 9: Score 0.7611


In [166]:
np.mean(np.array(preds_anom))

0.6376266095941842

In [167]:
b1

array([0.28766319, 0.10338632, 0.62942155, 0.28946381, 0.11335123,
       0.1094798 , 0.07664245, 0.29747496, 0.05860824])

Initial Weights

In [168]:
dt_norm = []

for i, clf in enumerate(clfs):
    dt_norm.append(clf.predict_proba(xs[i]))

In [169]:
hl1_this_all = np.zeros((len(df_oc_oh),len(cols[1:])))
preds = []

for j,x_prime in enumerate(dt_norm):
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys[j]]
        
        preds.append(neuron_l1(x_prime,w1_init[j],bias_init[j],num_labels))

In [170]:
for i,this_pred in enumerate(preds):
    print('Attribute {}: Score {}'.format(i+1,np.mean(this_pred)))

Attribute 1: Score 0.6912275403955102
Attribute 2: Score 0.7199679688193054
Attribute 3: Score 0.6202717966617182
Attribute 4: Score 0.6913042905024895
Attribute 5: Score 0.728418202635014
Attribute 6: Score 0.6907720864305099
Attribute 7: Score 0.63777535255288
Attribute 8: Score 0.4731615902507555
Attribute 9: Score 0.7214147839299194


In [171]:
np.mean(np.array(preds))

0.6638126235753448

In [172]:
hl1_this_anom = np.zeros((len(df_anom_oh),len(cols[1:])))
preds_anom = []

for j,x_prime in enumerate(dt_anom):
        
        label = cols[1:][j]
        num_labels = [map_dicts_v2[label][f] for f in ys_anom[j]]
        
        preds_anom.append(neuron_l1(x_prime,w1_init[j],bias_init[j],num_labels))

In [173]:
for i,this_pred in enumerate(preds_anom):
    print('Attribute {}: Score {}'.format(i+1,np.mean(this_pred)))

Attribute 1: Score 0.5332323086324527
Attribute 2: Score 0.5390414314870995
Attribute 3: Score 0.5093692540191332
Attribute 4: Score 0.6309784608815728
Attribute 5: Score 0.591314281673475
Attribute 6: Score 0.526431575470484
Attribute 7: Score 0.5153265172539562
Attribute 8: Score 0.5058281045548834
Attribute 9: Score 0.620899902587802


In [174]:
np.mean(np.array(preds_anom))

0.5524913151734288